# Глубокое обучение и обработка естественного языка

## Домашняя работа №1

1.   Загрузить набор данных по выбору с помощью библиотеки Corus - 1 балл
2.   Провести предобработку выбранного датасета: - 6 баллов

  *   Нормализация
  *   Токенизация
  *   Удаление стоп-слов
  *   Лемматизация/стемминг

3.   Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - 2 балла
4.   Соблюден code style на уровне pep8 и On writing clean Jupyter notebooks - 1 балл

Инструменты для решения задач предобработки – NLTK, Gensim, Natasha, pymystem, pymorphy2…

Для сдачи ДЗ - приложите ссылку на PR (Pull Request) из ветки hw_1 в ветку main в вашем приватном репозитории на github.com


In [ ]:
# установка набора данных
!pip install corus
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

# библиотека для очистки текста
!pip install clean-text

# библиотека для лемматизации слов
!pip install pymorphy2

# библиотека для проверки правописания слов
!pip install pyspellchecker

In [ ]:
# подключение библиотек
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from cleantext import clean
from corus import load_rudrec

nltk.download('punkt') # punkt tokenizer
nltk.download('stopwords')

### 1. Набор данных
RuDReC - это новый частично аннотированный корпус отзывов потребителей на русском языке о фармацевтической продукции для выявления названных объектов, связанных со здоровьем, и оценки эффективности фармацевтической продукции.

In [48]:
path = 'rudrec_annotated.json'
records = load_rudrec(path)

dataset = [record.text for record in records]
dataset[:10]

['нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
 'Общее впечатление : не подошел\n',
 'Пила этот препарат для повышения иммунитета 5 лет назад.\n',
 'Так как начала работать в аптеке, начала часто болеть.\n',
 'В месяц по нескольку раз причем со всеми вытекающими симптомами : кашель, боль горле, насморк.\n',
 'Принимала Иммунал строго по инструкции 4 раза в день 3 недели.\n',
 'Начала его принимать совершенно здоровой, через 1,5 недели после приема заболела.\n',
 'Вот и вывод препарат по идее должен был уже начать действовать так как минимальный курс его приема составляет 1 неделю, а заболела я на 11 день после начала приема.\n',
 'Пропив 5 упаковок это примерно 500-600 рублей пять лет назад, я как болела, так и продолжала болеть.\n',
 'Время использования: 3 недели\n']

### 2. Предобработка данных
#### 1. Нормализация текста

*   Приведение к нижнему регистру
*   Удаление пунктуации, переносов строк
*   Удаление url, номеров телефонов, email, чисел, цифр, символов валют


In [49]:
def clean_record(record):
  return clean(record,
                  fix_unicode=True,
                  to_ascii=False,
                  lower=True,
                  no_line_breaks=True,
                  no_urls=True,
                  no_emails=True,
                  no_phone_numbers=True,
                  no_numbers=True,
                  no_digits=True,
                  no_currency_symbols=True,
                  no_punct=True,
                  replace_with_punct="",
                  replace_with_url="",
                  replace_with_email="",
                  replace_with_phone_number="",
                  replace_with_number="",
                  replace_with_digit="",
                  replace_with_currency_symbol="",
                  lang="ru"
              )

In [50]:
dataset = [clean_record(record) for record in dataset]
dataset[:10]

['нам прописали так мой ребенок сыпью покрылся глаза опухли сверху и снизу на веках высыпала сыпь месяцев сынуа от виферона такого не было у кого ещё такие побочки отзовитесь чем спасались',
 'общее впечатление не подошел',
 'пила этот препарат для повышения иммунитета лет назад',
 'так как начала работать в аптеке начала часто болеть',
 'в месяц по нескольку раз причем со всеми вытекающими симптомами кашель боль горле насморк',
 'принимала иммунал строго по инструкции раза в день недели',
 'начала его принимать совершенно здоровой через недели после приема заболела',
 'вот и вывод препарат по идее должен был уже начать действовать так как минимальный курс его приема составляет неделю а заболела я на день после начала приема',
 'пропив упаковок это примерно рублей пять лет назад я как болела так и продолжала болеть',
 'время использования недели']

#### 2. Токенизация
Токенизация текста по словам

In [51]:
records_tokenized = [word_tokenize(record, language='russian') for record in dataset]
records_tokenized[:3]

[['нам',
  'прописали',
  'так',
  'мой',
  'ребенок',
  'сыпью',
  'покрылся',
  'глаза',
  'опухли',
  'сверху',
  'и',
  'снизу',
  'на',
  'веках',
  'высыпала',
  'сыпь',
  'месяцев',
  'сынуа',
  'от',
  'виферона',
  'такого',
  'не',
  'было',
  'у',
  'кого',
  'ещё',
  'такие',
  'побочки',
  'отзовитесь',
  'чем',
  'спасались'],
 ['общее', 'впечатление', 'не', 'подошел'],
 ['пила',
  'этот',
  'препарат',
  'для',
  'повышения',
  'иммунитета',
  'лет',
  'назад']]

#### 4. Удаление стоп-слов

In [52]:
stop_words = set(stopwords.words('russian'))
records_tokenized_filtered = []

for record in records_tokenized:
  record_filtered = []
  for word in record:
    if word not in stop_words:
        record_filtered.append(word)
  records_tokenized_filtered.append(record_filtered)

records_tokenized_filtered[:3]

[['нам',
  'прописали',
  'ребенок',
  'сыпью',
  'покрылся',
  'глаза',
  'опухли',
  'сверху',
  'снизу',
  'веках',
  'высыпала',
  'сыпь',
  'месяцев',
  'сынуа',
  'виферона',
  'такого',
  'кого',
  'ещё',
  'такие',
  'побочки',
  'отзовитесь',
  'спасались'],
 ['общее', 'впечатление', 'подошел'],
 ['пила', 'препарат', 'повышения', 'иммунитета', 'лет', 'назад']]

#### 5. Лемматизация
Приведение слов к начальной форме

In [53]:
morph = pymorphy2.MorphAnalyzer()
records_lemmatized = []

for record in records_tokenized_filtered:
  record_lemmatized = []
  for word in record:
    record_lemmatized.append(morph.parse(word)[0].normal_form)
  records_lemmatized.append(record_lemmatized)

records_lemmatized[:3]

[['мы',
  'прописать',
  'ребёнок',
  'сыпь',
  'покрыться',
  'глаз',
  'опухнуть',
  'сверху',
  'снизу',
  'век',
  'высыпать',
  'сыпь',
  'месяц',
  'сынуа',
  'виферон',
  'такой',
  'кто',
  'ещё',
  'такой',
  'побочка',
  'отозваться',
  'спасаться'],
 ['общий', 'впечатление', 'подойти'],
 ['пила', 'препарат', 'повышение', 'иммунитет', 'год', 'назад']]

#### 6. Проверка орфографии

*   Обнаружение опечаток
*   Ручное исправление
*   Удаление странных слов

Проверка орфографии проводится после лемматизации, поскольку spellchecker'у неизвестны некоторые формы слов, в результате чего он возвращает много результатов для исправления

In [54]:
mistakes = ['оганизм', 'свича', 'чюдо', 'сынуа', 'хмесяц', 'ужасс', 'таблеточк', 'прпарат', 'результая', 'маленкий', 'деньга', 'разэ', 'снимка',
            'вродя', 'основный', 'исспользовать', 'политься', 'интерессный', 'месяцый', 'поверот', 'выпиоа', 'преперат', 'растройство', 'прийтись',
            'саый', 'кафеин', 'боротьс', 'бориться', 'попрбовать', 'чють', 'счас', 'хочеться', 'энужный', 'выпоить', 'витаминк', 'многий', 'проводиться',
            'подругий', 'кстат', 'сторонка', 'нео', 'алергия', 'сли', 'порпросить', 'комплкс', 'производиться', 'температруа', 'видти', 'чуство', 'гдеть',
            'срашно', 'средсвт', 'продаеться', 'давалал', 'колличество', 'иполезный', ',эффектьивность', 'тмин', 'споровождение', 'чача',
            'приходиться', 'многооо', 'женщинец', 'хорошоя', 'вося', 'целиков', 'грипа', 'перу', 'дол', 'счастие', 'новерный', 'спусть', 'препорат', 'хотяб',
            'спакойный', 'повториться', 'бытро', 'брызга', 'продалжать', 'железа', 'ппробовать', 'получиться', 'терапиия', 'плозие', 'вобщий', 'зрр', 'удав',
            'эффекивность', 'мусорк', 'помогтут', 'едить', 'успееш', 'говориться', 'чтой', 'материа', 'родсознание', 'вседа', 'какао', 'проблемма', 'случий',
            'строп', 'жырный', 'умнас', 'мложно', 'хота', 'изз', 'говня', 'конечнож', 'порою', 'побочый', ]

to_delete = ['доко', 'кус', 'ранон', 'чмт', 'р', 'квд', 'c', 'п', 'атечка', 'пр', 'b', 'африна', 's', 'ый', 'б', 'л', 'з', 'сее', 'иня', 'неный', 'носов',
             'никочь', 'частность', 'ивот', 'минь', 'троить', 'мышщи', 'всд', 'х', 'ходиш', 'гв', 'к', 'грип', 'ставт', 'дв', 'альфаb', 'кор', 'позный',
             'оч', '№', 'ёх', 'ор', 'физ', 'ношп', 'дада', 'ребен', 'срей', 'толя', 'дмн', 'барра', 'лайта', 'чбд', 'иса', 'суткин', 'эдем', 'ть', 'лучить',
             'ве', 'аш', 'чтоять', 'млг', 'й', 'рр', 'ирс', 'грн', 'киев', 'блистер', 'ринза', 'питься']

In [55]:
spell = SpellChecker(distance=1, language='ru')

for record in records_lemmatized:
  for i in range(len(record)):
    if record[i] in mistakes:
      record[i] = spell.correction(record[i])

  for word in to_delete:
    if word in record:
      record.remove(word)

records_lemmatized[:3]

[['мы',
  'прописать',
  'ребёнок',
  'сыпь',
  'покрыться',
  'глаз',
  'опухнуть',
  'сверху',
  'снизу',
  'век',
  'высыпать',
  'сыпь',
  'месяц',
  'сына',
  'виферон',
  'такой',
  'кто',
  'ещё',
  'такой',
  'побочка',
  'отозваться',
  'спасаться'],
 ['общий', 'впечатление', 'подойти'],
 ['пила', 'препарат', 'повышение', 'иммунитет', 'год', 'назад']]

Повторное проведение лемматизации, тк spellchecker мог предложить в качестве исправления не начальную форму слова

In [56]:
records_lemmatized_clear = []

for record in records_lemmatized:
  record_lemmatized = []
  for word in record:
    record_lemmatized.append(morph.parse(word)[0].normal_form)
  records_lemmatized_clear.append(record_lemmatized)

records_lemmatized_clear[:3]

[['мы',
  'прописать',
  'ребёнок',
  'сыпь',
  'покрыться',
  'глаз',
  'опухнуть',
  'сверху',
  'снизу',
  'век',
  'высыпать',
  'сыпь',
  'месяц',
  'сын',
  'виферон',
  'такой',
  'кто',
  'ещё',
  'такой',
  'побочка',
  'отозваться',
  'спасаться'],
 ['общий', 'впечатление', 'подойти'],
 ['пила', 'препарат', 'повышение', 'иммунитет', 'год', 'назад']]

#### Зависимости

In [57]:
! pip freeze > requirements.txt